# Importación de librerías

In [1]:
import sys
sys.path.append('../')

from src.func_aux_cities import *

# Segundo intento de hacer el plot temporal del mapa

In [2]:
#!pip install netCDF4

In [3]:
import xarray as xr
import cftime
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset

In [4]:
map_df=xr.open_dataset('../data/sea_level.nc')

map_df

<xarray.Dataset>
Dimensions:                 (time: 276, lat: 720, lon: 1440)
Coordinates:
  * time                    (time) object 1993-01-15 00:00:00 ... 2015-12-15 ...
  * lat                     (lat) float32 -89.88 -89.62 -89.38 ... 89.62 89.88
  * lon                     (lon) float32 0.125 0.375 0.625 ... 359.6 359.9
Data variables:
    global_msl              (time) float32 ...
    global_msl_trend        float32 ...
    global_msl_trend_error  float32 ...
Attributes: (12/35)
    title:                     Mean Sea Level temporal variations
    institution:               ESA, CLS, CNES
    references:                http://www.esa-sealevel-cci.org/products
    tracking_id:               
    source:                    ERS-1 Phase C OPR V6, ERS-1 Phase E OPR V3, ER...
    platform:                  ERS-1, ERS-2, ENVISAT, TOPEX/Poseidon, Jason-1...
    ...                        ...
    geospatial_vertical_min:   0.0
    geospatial_vertical_max:   0.0
    time_coverage_start:       1993-01-01 00:00:00
    time_coverage_end:         2015-12-31 23:59:59
    time_coverage_duration:    P23Y
    time_coverage_resolution:  P1M

In [5]:
map_df.keys()

KeysView(<xarray.Dataset>
Dimensions:                 (time: 276, lat: 720, lon: 1440)
Coordinates:
  * time                    (time) object 1993-01-15 00:00:00 ... 2015-12-15 ...
  * lat                     (lat) float32 -89.88 -89.62 -89.38 ... 89.62 89.88
  * lon                     (lon) float32 0.125 0.375 0.625 ... 359.6 359.9
Data variables:
    global_msl              (time) float32 -0.001873 -0.004756 ... 0.07723
    global_msl_trend        float32 2.978
    global_msl_trend_error  float32 0.02354
Attributes: (12/35)
    title:                     Mean Sea Level temporal variations
    institution:               ESA, CLS, CNES
    references:                http://www.esa-sealevel-cci.org/products
    tracking_id:               
    source:                    ERS-1 Phase C OPR V6, ERS-1 Phase E OPR V3, ER...
    platform:                  ERS-1, ERS-2, ENVISAT, TOPEX/Poseidon, Jason-1...
    ...                        ...
    geospatial_vertical_min:   0.0
    geospatial_vert

In [6]:
climate=map_df.resample(time='Y').mean()

anomaly=climate['global_msl']

anomaly

<xarray.DataArray 'global_msl' (time: 23)>
array([0.00031873, 0.00085513, 0.00326519, 0.00631262, 0.01045041,
       0.01395107, 0.01562467, 0.01952518, 0.02370682, 0.0262177 ,
       0.02826822, 0.03056715, 0.03530574, 0.03717059, 0.03713357,
       0.0417086 , 0.04487981, 0.04628378, 0.04629683, 0.05614512,
       0.05814245, 0.06113304, 0.0702509 ], dtype=float32)
Coordinates:
  * time     (time) object 1993-12-31 00:00:00 ... 2015-12-31 00:00:00

In [7]:
df = anomaly.to_dataframe()
df.head()

,global_msl
time,
1993-12-31 00:00:00,0.000319
1994-12-31 00:00:00,0.000855
1995-12-31 00:00:00,0.003265
1996-12-31 00:00:00,0.006313
1997-12-31 00:00:00,0.010450


In [8]:
df.reset_index(inplace=True)
df.head() #23x3

,time,global_msl
0,1993-12-31 00:00:00,0.000319
1,1994-12-31 00:00:00,0.000855
2,1995-12-31 00:00:00,0.003265
3,1996-12-31 00:00:00,0.006313
4,1997-12-31 00:00:00,0.010450


In [9]:
df2 = pd.DataFrame()
df2['lon'] = list(pd.Series(map_df.variables['lon'][:]))
df2['lon'] = df2.lon.apply(lambda x:[x]*len(map_df.variables['lat'][:]))
df2.head()

,lon
0,"[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.1..."
1,"[0.375, 0.375, 0.375, 0.375, 0.375, 0.375, 0.3..."
2,"[0.625, 0.625, 0.625, 0.625, 0.625, 0.625, 0.6..."
3,"[0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.8..."
4,"[1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.1..."


In [10]:
lst = list(df2.lon)

In [11]:
def desenredar(lst):
    res = []
    
    for i in lst:
        for j in i:
            res.append(j)
    
    return res

lst_def = desenredar(lst)

In [12]:
df_def = pd.DataFrame()
df_def['lon'] = lst_def
df_def.head()

,lon
0,0.125
1,0.125
2,0.125
3,0.125
4,0.125


In [13]:
df_def.shape

(1036800, 1)

In [14]:
lats = list(pd.Series(map_df.variables['lat'][:]))
lats = lats*len(list(map_df.variables['lon'][:]))
df_def['lat'] = lats
df_def.head()

,lon,lat
0,0.125,-89.875
1,0.125,-89.625
2,0.125,-89.375
3,0.125,-89.125
4,0.125,-88.875


In [15]:
df_def.shape

(1036800, 2)

In [24]:
df.time = df.time.apply(lambda x:[x]*720*1440)

In [21]:
23846400/(720*1440)

23.0

In [22]:
#tenemos que en cada celda del dataframe hay que meterla para el dataframe de cada año

## Dataframe de 1993

In [ ]:
df1993 = pd.DataFrame()
df1993['Time'] = df.time[0]
df1993['Lat'] = df_def.lat
df1993['Lon'] = df_def.lon
df1993.head()